In [1]:
import os
import pickle
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
re.compile('(.*)')
preProcessLemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path='C:/Users/Asus/AssignmentTest/text_files_fresh'
list1=os.listdir(path)
for i in range(1,5):
    print(list1[i])

file10.txt
file100.txt
file101.txt
file102.txt


In [3]:
#initalizing empty dictonary to store invert index
index = {}
# Process each file in the directory
for filename in list1:
    with open(os.path.join(path, filename), "r") as file:
        # Read file contents
        fileContent = file.read()
        #Tokenize each word from the open file
        Tkens = word_tokenize(fileContent)

         # add postings to the dictionary index
        for token in Tkens:
            if token not in index: #token not present then initialize new 
                index[token] = [] 
            if filename not in index[token]:
                index[token].append(filename)
# Iterate over the keys of the index dictionary, limiting to 5 items
for i, token in enumerate(index.keys()):
    if i == 5:
        break
    # Print the token and its associated postings list
    print(f"{token}: {index[token]}")
    print('')



loving: ['file1.txt', 'file254.txt', 'file391.txt', 'file723.txt']

vintage: ['file1.txt', 'file150.txt', 'file197.txt', 'file278.txt', 'file422.txt', 'file439.txt', 'file494.txt', 'file51.txt', 'file597.txt', 'file638.txt', 'file674.txt', 'file725.txt', 'file737.txt', 'file827.txt', 'file847.txt', 'file895.txt', 'file907.txt', 'file936.txt']

springs: ['file1.txt', 'file272.txt', 'file469.txt', 'file806.txt', 'file937.txt']

strat: ['file1.txt', 'file149.txt', 'file163.txt', 'file197.txt', 'file241.txt', 'file245.txt', 'file25.txt', 'file253.txt', 'file345.txt', 'file353.txt', 'file380.txt', 'file396.txt', 'file400.txt', 'file422.txt', 'file440.txt', 'file455.txt', 'file457.txt', 'file469.txt', 'file519.txt', 'file529.txt', 'file559.txt', 'file565.txt', 'file579.txt', 'file611.txt', 'file626.txt', 'file650.txt', 'file652.txt', 'file691.txt', 'file725.txt', 'file801.txt', 'file838.txt', 'file853.txt', 'file90.txt', 'file940.txt', 'file978.txt', 'file993.txt']

good: ['file1.txt', 'file

In [4]:
# Save postings to file
with open('index.pkl', 'wb') as f:
    pickle.dump(index, f)

In [5]:
file_IdMap={}
i=0
doc_names = ""
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        file_IdMap[i]=filename
        i += 1
for i, token in enumerate(file_IdMap.keys()):
    if i == 5:
        break
    # Print the token and its associated postings list
    print(f"{token}: {file_IdMap[token]}")
    print('')    

0: file1.txt

1: file10.txt

2: file100.txt

3: file101.txt

4: file102.txt



In [6]:
with open('fileMapping', 'wb') as fh:
   pickle.dump(file_IdMap, fh)

In [7]:
def preprocess(input_text):
    # lowercase
    text_lower = input_text.lower()
    
    # Tokenize
    text_tokens = word_tokenize(text_lower)
    
    #exclusion of stopwords from the string and removal of puncations
    swords = stopwords.words('english')
    tokenstopwords = [token for token in text_tokens if token not in swords]
    stopwordsRemoved = ' '.join(tokenstopwords)
    text_no_punc = re.sub(r'[^\w\s]', '', stopwordsRemoved)
    
    #removing extra spaces 
    text_cleaned = text_no_punc.replace("  ", " ")
    return text_cleaned

In [8]:
def merge_lists(set1, set2, operation):
    set_result = set()
    universal_set = set(file_IdMap.values())
    if operation == 'AND':
        set_result = set1.intersection(set2)
      
    elif operation == 'AND NOT':
        set_result = set1.difference(set2)
       
    elif operation == 'OR':
        set_result = set1.union(set2)
       
    elif operation == 'OR NOT':
        set_result = set1.union(set1, universal_set - set2)
       

    return set_result

In [13]:
def queryProcess(query, operators, index):
    fileNames = []

    for i, op in enumerate(operators):
        op = op.strip()
        if i == 0:
            word1= index[query[i]] if query[i] in index else []
            word2= index[query[i+1]] if query[i+1] in index else []


            if op == "AND":
                fileNames= merge_lists(set(word1.copy()), set(word2.copy()), 'AND')
            elif op == "OR":
                fileNames= merge_lists(set(word1.copy()), set(word2.copy().copy()), 'OR')
            elif op == "AND NOT":
                fileNames= merge_lists(set(word1.copy()), set(word2.copy().copy()), 'AND NOT')
            elif op == "OR NOT":
                fileNames= merge_lists(set(word1.copy()), set(word2.copy().copy()), 'OR NOT')
            else:
                print("Invalid Query!")
                return None
        else:
            if query[i + 1] in index:
                word2 = index[query[i + 1]]
            else:
                word2 = []
            if op == "AND":
                fileNames= merge_lists(set(fileNames.copy()), set(word2.copy()), 'AND')
            elif op == "OR":
                fileNames= merge_lists(set(fileNames.copy()), set(word2.copy()), 'OR')
            elif op == "AND NOT":
                fileNames= merge_lists(set(fileNames.copy()), set(word2.copy()), 'AND NOT')
            elif op == "OR NOT":
                fileNames = merge_lists(set(fileNames.copy()), set(word2.copy()), 'OR NOT')
            else:
                print("Invalid Query")
                return None
        

    return fileNames

In [14]:
N = int(input("Enter the number of queries: "))  # number of queries
queries = []
operator_list = []

# Get queries and operators from the user
for p in range(N):
    print("Query ", p + 1, ":")
    x = input("Enter input sequence: ")
    queries.append(x)
    x = input("Enter operation sequence: ")
    operator_list.append(x)

# Process each query
for k in range(N):  # for each query
    query = preprocess(queries[k])
    operators = operator_list[k]
    operators = operators.upper().split(",")  # get operators
    query = word_tokenize(query)
    print(query)
    print(operators)

    print("Query ",k+1,":",end=" ")

    if(len(query) != (len(operators)+1)):
        print("Invalid query")
    else:
        for i in range(len(query)-1):
            print(query[i]+" "+operators[i]+" ",end="")
        print(query[len(query)-1])
        fileNames =queryProcess(query, operators,index)
        print("Number of documents retrieved for query ",k+1,": ", len(fileNames))

        print("Names of the documents retrieved for query ", k + 1, ": ", fileNames)


Enter the number of queries: 1
Query  1 :
Enter input sequence: power supply
Enter operation sequence: AND
['power', 'supply']
['AND']
Query  1 : power AND supply
Number of documents retrieved for query  1 :  14
Names of the documents retrieved for query  1 :  {'file367.txt', 'file434.txt', 'file512.txt', 'file222.txt', 'file521.txt', 'file597.txt', 'file687.txt', 'file760.txt', 'file988.txt', 'file19.txt', 'file305.txt', 'file828.txt', 'file55.txt', 'file223.txt'}
